# Imports

In [ ]:
import torch
import torch.optim as optim
import numpy as np
import node_models
import loader
import training
import metrics
import autotune
import config

# GPU

In [ ]:
device = torch.device(1)

# Data Loading

In [ ]:
cifar_data_path = './cifar-10-batches-py'

In [ ]:
cifar_data = loader.CIFAR(cifar_data_path, 3)
cifar_splits = cifar_data.splits

In [ ]:
print("CIFAR STATS")
print("Number of splits:", len(cifar_splits))
print("Number of segments per split (train, val, test):", len(cifar_splits[0]))
print("Info per segment (data, labels):", len(cifar_splits[0][0]))
print("Size of segement (num examples):", len(cifar_splits[0][0][0]))

# Model & Optimizer

In [ ]:
model = node_models.NeuralODE
optimizer = optim.Adam

# Training and Evaluation

## CIFAR

### NODE

In [ ]:
model_params = config.baseline_config_cifar_node['model']
lr, epochs, batch, workers = config.baseline_config_cifar_node['train']

In [ ]:
node_cifar_trainer = training.Trainer(model, optimizer, cifar_data, device)

In [ ]:
node_cifar_trainer.train(model_params, lr, epochs, batch, num_workers=6, verbose=True, num_loss=3)

In [ ]:
node_cifar_trainer.test(model_params, 10, 3)

In [ ]:
 torch.cuda.empty_cache()

### ANODE

In [ ]:
model_params = config.baseline_config_cifar_anode['model']
lr, epochs, batch, workers = config.baseline_config_cifar_anode['train']

In [ ]:
anode_cifar_trainer = training.Trainer(model, optimizer, cifar_data, device)

In [ ]:
anode_cifar_trainer.train(model_params, lr, epochs, batch, num_workers=6, verbose=True, num_loss=3)

In [ ]:
anode_cifar_trainer.test(model_params, 10, 3)

In [ ]:
print(anode_cifar_trainer.val_metrics['loss'])

# Plots

In [ ]:
node_cifar_trainer.val_metrics['legend'] = 'NODE'
anode_cifar_trainer.val_metrics['legend'] = 'ANODE'
out_metrics = [node_cifar_trainer.val_metrics, anode_cifar_trainer.val_metrics]

In [ ]:
plt = metrics.Plotter(out_metrics)

In [ ]:
plt.plotLoss("Optimized Model Validation Loss Comparisons")

In [ ]:
plt.plotAccuracy("Optimized Model Validation Accuracy Comparisons")

In [ ]:
plt.plotNFE("Loss vs NFE", style='loss')

In [ ]:
plt.plotNFE("Accuracy vs NFE", style='accuracy')